In [ ]:
import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import panel as pn

In [ ]:
import sqlite3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/clinicaa.db')
cursor = conn.cursor()

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS funcionario (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    telefone_1 TEXT NOT NULL,
    telefone_2 TEXT,
    logradouro TEXT NOT NULL,
    numero TEXT NOT NULL,
    complemento TEXT,
    bairro TEXT NOT NULL,
    cidade TEXT NOT NULL,
    uf TEXT NOT NULL,
    cep TEXT NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS medico (
    medico_id INTEGER PRIMARY KEY,
    crm TEXT UNIQUE NOT NULL,
    FOREIGN KEY (medico_id) REFERENCES funcionario(id) ON DELETE CASCADE ON UPDATE CASCADE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS especialidade (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS medico_especialidade (
    medico_id INTEGER NOT NULL,
    especialidade_id INTEGER NOT NULL,
    PRIMARY KEY (medico_id, especialidade_id),
    FOREIGN KEY (medico_id) REFERENCES medico(medico_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (especialidade_id) REFERENCES especialidade(id) ON DELETE CASCADE ON UPDATE CASCADE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS secretario (
    registro_id INTEGER PRIMARY KEY,
    funcao TEXT NOT NULL,
    FOREIGN KEY (registro_id) REFERENCES funcionario(id) ON DELETE CASCADE ON UPDATE CASCADE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS insumos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL UNIQUE,
    quantidade INTEGER NOT NULL,
    validade DATE NOT NULL,
    data_reposicao DATE NOT NULL,
    secretario_id INTEGER NOT NULL,
    FOREIGN KEY (secretario_id) REFERENCES secretario(registro_id) ON DELETE SET NULL ON UPDATE CASCADE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS categoria (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS insumo_categoria (
    insumo_id INTEGER NOT NULL,
    categoria_id INTEGER NOT NULL,
    PRIMARY KEY (insumo_id, categoria_id),
    FOREIGN KEY (insumo_id) REFERENCES insumos(id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (categoria_id) REFERENCES categoria(id) ON DELETE CASCADE ON UPDATE CASCADE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS paciente (
    cpf TEXT PRIMARY KEY,
    nome TEXT NOT NULL,
    email TEXT,
    telefone TEXT NOT NULL,
    logradouro TEXT NOT NULL,
    numero TEXT NOT NULL,
    complemento TEXT,
    bairro TEXT NOT NULL,
    cidade TEXT NOT NULL,
    uf TEXT NOT NULL,
    cep TEXT NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS consultas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    data_ DATE NOT NULL,
    horas TIME NOT NULL,
    status TEXT NOT NULL,
    medico_id INTEGER NOT NULL,
    paciente_id TEXT NOT NULL,
    FOREIGN KEY (medico_id) REFERENCES medico(medico_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (paciente_id) REFERENCES paciente(cpf) ON DELETE CASCADE ON UPDATE CASCADE
)
''')

In [ ]:
# Inserir dados na tabela de funcionário
funcionarios = [
    ('João Silva', 'joao.silva@example.com', '1234567890', '0987654321', 'Rua A', '123', 'Apto 101', 'Centro', 'São Paulo', 'SP', '00000-000'),
    ('Maria Oliveira', 'maria.oliveira@example.com', '2345678901', '1987654321', 'Rua B', '456', '', 'Jardim', 'Rio de Janeiro', 'RJ', '11111-111'),
    ('Pedro Santos', 'pedro.santos@example.com', '3456789012', '2987654321', 'Rua C', '789', 'Casa 303', 'Vila', 'Belo Horizonte', 'MG', '22222-222'),
    ('Ana Costa', 'ana.costa@example.com', '4567890123', '3987654321', 'Rua D', '101', 'Apto 404', 'Bela Vista', 'Salvador', 'BA', '33333-333'),
    ('Carlos Almeida', 'carlos.almeida@example.com', '5678901234', '4987654321', 'Rua E', '202', '', 'Lagoa', 'Curitiba', 'PR', '44444-444'),
    ('Patrícia Lima', 'patricia.lima@example.com', '6789012345', '5987654321', 'Rua F', '303', 'Apto 606', 'Praia', 'Porto Alegre', 'RS', '55555-555'),
    ('Fernanda Rocha', 'fernanda.rocha@example.com', '7890123456', '6987654321', 'Rua G', '404', 'Casa 707', 'Campo', 'Florianópolis', 'SC', '66666-666'),
    ('Roberto Martins', 'roberto.martins@example.com', '8901234567', '7987654321', 'Rua H', '505', 'Apto 808', 'Serra', 'Vitória', 'ES', '77777-777'),
    ('Juliana Ferreira', 'juliana.ferreira@example.com', '9012345678', '8987654321', 'Rua I', '606', 'Casa 909', 'Zona Norte', 'Goiânia', 'GO', '88888-888'),
    ('Lucas Almeida', 'lucas.almeida@example.com', '0123456789', '9987654321', 'Rua J', '707', 'Apto 010', 'Zona Sul', 'Cuiabá', 'MT', '99999-999')
]

cursor.executemany('''
INSERT INTO funcionario (nome, email, telefone_1, telefone_2, logradouro, numero, complemento, bairro, cidade, uf, cep)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', funcionarios)

In [ ]:
# Inserir dados na tabela de médico
medicos = [
    (11, 'CRM12345'), # Dr. Paulo Andrade
    (12, 'CRM12346'), # Dra. Clara Souza
    (13, 'CRM12347'), # Dr. Eduardo Lima
    (14, 'CRM12348'), # Dra. Helena Borges
    (15, 'CRM12349'), # Dr. Marcos Tavares
    (16, 'CRM12350'), # Dra. Aline Castro
    (17, 'CRM12351'), # Dr. Ricardo Mendes
    (18, 'CRM12352'), # Dra. Beatriz Dias
    (19, 'CRM12353'), # Dr. Fernando Gomes
    (20, 'CRM12354')  # Dra. Camila Azevedo
]

cursor.executemany('''
INSERT INTO medico (medico_id, crm)
VALUES (?, ?)
''', medicos)

In [ ]:
# Inserir dados na tabela de secretário
secretarios = [
    (1, 'Secretário de Recepção'), # João Silva
    (2, 'Secretário de Consultas'), # Maria Oliveira
    (3, 'Secretário Administrativo'), # Pedro Santos
    (4, 'Secretário de Atendimento'), # Ana Costa
    (5, 'Secretário de Atendimento Médico'), # Carlos Almeida
    (6, 'Secretário de Agendamento'), # Patrícia Lima
    (7, 'Secretário de Documentação'), # Fernanda Rocha
    (8, 'Secretário de Registros'), # Roberto Martins
    (9, 'Secretário de Triagem'), # Juliana Ferreira
    (10, 'Secretário de Comunicação') # Lucas Almeida
]

cursor.executemany('''
INSERT INTO secretario (registro_id, funcao)
VALUES (?, ?)
''', secretarios)

In [ ]:
for secretarios in cursor.execute('SELECT * FROM secretario'):
    print(secretarios)

(1, 'Secretário de Recepção')
(2, 'Secretário de Consultas')
(3, 'Secretário Administrativo')
(4, 'Secretário de Atendimento')
(5, 'Secretário de Atendimento Médico')
(6, 'Secretário de Agendamento')
(7, 'Secretário de Documentação')
(8, 'Secretário de Registros')
(9, 'Secretário de Triagem')
(10, 'Secretário de Comunicação')


In [ ]:
# Inserir dados na tabela de insumos
insumos = [
    ('Gaze Estéril', 500, '2024-12-31', '2024-11-01', 1),
    ('Algodão Hidrófilo', 300, '2025-01-15', '2024-12-01', 2),
    ('Seringa Descartável 5ml', 1000, '2025-06-20', '2025-05-01', 3),
    ('Luvas Cirúrgicas', 200, '2025-03-30', '2025-02-01', 4),
    ('Agulha Hipodérmica', 1500, '2025-02-28', '2025-01-10', 5),
    ('Esparadrapo Microporoso', 250, '2025-08-30', '2025-07-01', 6),
    ('Compressa de Gaze', 400, '2025-04-15', '2025-03-01', 7),
    ('Soro Fisiológico 500ml', 50, '2024-11-30', '2024-10-15', 8),
    ('Curativo Adesivo', 300, '2024-10-20', '2024-09-01', 9),
    ('Termômetro Digital', 20, '2026-01-01', '2025-12-01', 10)
]

cursor.executemany('''
INSERT INTO insumos (nome, quantidade, validade, data_reposicao, secretario_id)
VALUES (?, ?, ?, ?, ?)
''', insumos)

In [ ]:
# Inserir dados na tabela de especialidade
especialidades = [
    ('Cardiologia',),
    ('Neurologia',),
    ('Ortopedia',),
    ('Pediatria',),
    ('Oftalmologia',),
    ('Dermatologia',),
    ('Ginecologia',),
    ('Otorinolaringologia',),
    ('Psiquiatria',),
    ('Urologia',)
]

cursor.executemany('''
INSERT INTO especialidade (nome)
VALUES (?)
''', especialidades)

In [ ]:
# Inserir dados na tabela de medico_especialidade
medico_especialidades = [
    (11, 1),  # Dr. Paulo Andrade é Cardiologista
    (12, 2),  # Dra. Clara Souza é Neurologista
    (13, 3),  # Dr. Eduardo Lima é Ortopedista
    (14, 4),  # Dra. Helena Borges é Pediatra
    (15, 5),  # Dr. Marcos Tavares é Oftalmologista
    (16, 6),  # Dra. Aline Castro é Dermatologista
    (17, 7),  # Dr. Ricardo Mendes é Ginecologista
    (18, 8),  # Dra. Beatriz Dias é Otorinolaringologista
    (19, 9),  # Dr. Fernando Gomes é Psiquiatra
    (20, 10)  # Dra. Camila Azevedo é Urologista
]

cursor.executemany('''
INSERT INTO medico_especialidade (medico_id, especialidade_id)
VALUES (?, ?)
''', medico_especialidades)

In [ ]:
# Inserir dados na tabela de categoria
categorias = [
    ('Material de Curativo',),
    ('Material Cirúrgico',),
    ('Medicamento',),
    ('Equipamento',),
    ('Material de Limpeza',),
    ('Higiene Pessoal',),
    ('Instrumento Médico',),
    ('Produtos de Diagnóstico',),
    ('Materiais para Exames',),
    ('Acessórios Médicos',)
]

cursor.executemany('''
INSERT INTO categoria (nome)
VALUES (?)
''', categorias)

In [ ]:
# Inserir dados na tabela de insumo_categoria
insumo_categorias = [
    (1, 1),  # Gaze Estéril pertence a Material de Curativo
    (2, 1),  # Algodão Hidrófilo pertence a Material de Curativo
    (3, 2),  # Seringa Descartável 5ml pertence a Material Cirúrgico
    (4, 2),  # Luvas Cirúrgicas pertencem a Material Cirúrgico
    (5, 2),  # Agulha Hipodérmica pertence a Material Cirúrgico
    (6, 1),  # Esparadrapo Microporoso pertence a Material de Curativo
    (7, 1),  # Compressa de Gaze pertence a Material de Curativo
    (8, 3),  # Soro Fisiológico 500ml pertence a Medicamento
    (9, 1),  # Curativo Adesivo pertence a Material de Curativo
    (10, 4)  # Termômetro Digital pertence a Equipamento
]

cursor.executemany('''
INSERT INTO insumo_categoria (insumo_id, categoria_id)
VALUES (?, ?)
''', insumo_categorias)

In [ ]:
# Inserir dados na tabela de paciente
pacientes = [
    ('12345678901', 'Lucas Pereira', 'lucas.pereira@example.com', '11987654321', 'Rua A', '123', 'Apto 101', 'Centro', 'São Paulo', 'SP', '00000-000'),
    ('23456789012', 'Fernanda Lima', 'fernanda.lima@example.com', '21987654321', 'Rua B', '456', '', 'Jardim', 'Rio de Janeiro', 'RJ', '11111-111'),
    ('34567890123', 'Ricardo Silva', 'ricardo.silva@example.com', '31987654321', 'Rua C', '789', 'Casa 303', 'Vila', 'Belo Horizonte', 'MG', '22222-222'),
    ('45678901234', 'Juliana Costa', 'juliana.costa@example.com', '71987654321', 'Rua D', '101', 'Apto 404', 'Bela Vista', 'Salvador', 'BA', '33333-333'),
    ('56789012345', 'Roberta Santos', 'roberta.santos@example.com', '41987654321', 'Rua E', '202', 'Casa 505', 'Lagoa', 'Curitiba', 'PR', '44444-444'),
    ('67890123456', 'Eduardo Martins', 'eduardo.martins@example.com', '51987654321', 'Rua F', '303', 'Apto 606', 'Praia', 'Porto Alegre', 'RS', '55555-555'),
    ('78901234567', 'Tatiane Almeida', 'tatiane.almeida@example.com', '61987654321', 'Rua G', '404', 'Casa 707', 'Campo', 'Florianópolis', 'SC', '66666-666'),
    ('89012345678', 'Ricardo Ferreira', 'ricardo.ferreira@example.com', '71987654321', 'Rua H', '505', 'Apto 808', 'Serra', 'Vitória', 'ES', '77777-777'),
    ('90123456789', 'Camila Castro', 'camila.castro@example.com', '81987654321', 'Rua I', '606', 'Casa 909', 'Zona Norte', 'Goiânia', 'GO', '88888-888'),
    ('01234567890', 'Rodrigo Lima', 'rodrigo.lima@example.com', '91987654321', 'Rua J', '707', 'Apto 010', 'Zona Sul', 'Cuiabá', 'MT', '99999-999')
]

cursor.executemany('''
INSERT INTO paciente (cpf, nome, email, telefone, logradouro, numero, complemento, bairro, cidade, uf, cep)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', pacientes)

In [ ]:
# Inserir dados na tabela de consultas
consultas = [
    ('2024-09-15', '09:00:00', 'Agendada', 11, '12345678901'),  # Consulta com Dr. Paulo Andrade (Cardiologista) para Lucas Pereira
    ('2024-09-16', '10:00:00', 'Concluída', 12, '23456789012'),  # Consulta com Dra. Clara Souza (Neurologista) para Fernanda Lima
    ('2024-09-17', '11:00:00', 'Cancelada', 13, '34567890123'),  # Consulta com Dr. Eduardo Lima (Ortopedista) para Ricardo Silva
    ('2024-09-18', '14:00:00', 'Em Atendimento', 14, '45678901234'),  # Consulta com Dra. Helena Borges (Pediatra) para Juliana Costa
    ('2024-09-19', '15:00:00', 'Agendada', 15, '56789012345'),  # Consulta com Dr. Marcos Tavares (Oftalmologista) para Roberta Santos
    ('2024-09-20', '16:00:00', 'Concluída', 16, '67890123456'),  # Consulta com Dra. Aline Castro (Dermatologista) para Eduardo Martins
    ('2024-09-21', '17:00:00', 'Em Atendimento', 17, '78901234567'),  # Consulta com Dr. Ricardo Mendes (Ginecologista) para Tatiane Almeida
    ('2024-09-22', '09:00:00', 'Agendada', 18, '89012345678'),  # Consulta com Dra. Beatriz Dias (Otorinolaringologista) para Ricardo Ferreira
    ('2024-09-23', '10:00:00', 'Cancelada', 19, '90123456789'),  # Consulta com Dr. Fernando Gomes (Psiquiatra) para Camila Castro
    ('2024-09-24', '11:00:00', 'Concluída', 20, '01234567890')  # Consulta com Dra. Camila Azevedo (Urologista) para Rodrigo Lima
]

cursor.executemany('''
INSERT INTO consultas (data_, horas, status, medico_id, paciente_id)
VALUES (?, ?, ?, ?, ?)
''', consultas)

In [ ]:
# Confirmar as alterações
conn.commit()

# Fechar a conexão
conn.close()

In [ ]:
!pip install pandas psycopg2-binary sqlalchemy panel


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Caminho para o banco de dados no Google Drive
db_path = '/content/drive/MyDrive/clinicaa.db'
db_url = f'sqlite:///{db_path}'
engine = create_engine(db_url)

In [ ]:
# Carregar dados da tabela insumos
df_insumos = pd.read_sql('SELECT * FROM insumos', engine)

In [ ]:
!pip install jupyter_bokeh


In [ ]:
# Certifique-se de que o Panel está configurado para rodar no Jupyter
pn.extension()

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3 # importa o módulo sqlite3

# Caminho para o banco de dados no Google Drive
db_path = '/content/drive/MyDrive/clinicaa.db'

# Conecta ao banco de dados
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for insumos in cursor.execute('SELECT * FROM insumos'):
    print(insumos)


(1, 'Gaze Estéril', 500, '2024-12-31', '2024-11-01', 1)
(2, 'Algodão Hidrófilo', 300, '2025-01-15', '2024-12-01', 2)
(3, 'Seringa Descartável 5ml', 1000, '2025-06-20', '2025-05-01', 3)
(4, 'Luvas Cirúrgicas', 200, '2025-03-30', '2025-02-01', 4)
(5, 'Agulha Hipodérmica', 1500, '2025-02-28', '2025-01-10', 5)
(6, 'Esparadrapo Microporoso', 250, '2025-08-30', '2025-07-01', 6)
(7, 'Compressa de Gaze', 400, '2025-04-15', '2025-03-01', 7)
(8, 'Soro Fisiológico 500ml', 50, '2024-11-30', '2024-10-15', 8)
(9, 'Curativo Adesivo', 300, '2024-10-20', '2024-09-01', 9)
(10, 'Termômetro Digital', 20, '2026-01-01', '2025-12-01', 10)


In [ ]:
pn.extension()
output.enable_custom_widget_manager()

In [ ]:
!pip install jupyter_bokeh

In [203]:
import sqlite3
import pandas as pd
import panel as pn
from datetime import datetime
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine

# Habilitar a extensão do Panel
pn.extension()

# Conectar ao banco de dados com timeout
from google.colab import drive
drive.mount('/content/drive')
conn = sqlite3.connect('/content/drive/MyDrive/clinicaa.db', timeout=30)
cursor = conn.cursor()

# Função para adicionar insumo
def adicionar_insumo(nome, quantidade, secretario_id):
    try:
        with conn:
            cursor.execute("INSERT INTO insumos (nome, quantidade, secretario_id) VALUES (?, ?, ?)", (nome, quantidade, secretario_id))
    except sqlite3.OperationalError as e:
        print(f"Erro operacional: {e}")

# Função para entrada de insumo
def entrada_insumo(id, quantidade):
    try:
        with conn:
            cursor.execute("UPDATE insumos SET quantidade = quantidade + ? WHERE id = ?", (quantidade, id))
    except sqlite3.OperationalError as e:
        print(f"Erro operacional: {e}")

# Função para saída de insumo
def saida_insumo(id, quantidade):
    try:
        with conn:
            cursor.execute("UPDATE insumos SET quantidade = quantidade - ? WHERE id = ?", (quantidade, id))
    except sqlite3.OperationalError as e:
        print(f"Erro operacional: {e}")

# Função para atualizar tabela
def atualizar_tabela():
    try:
        df = pd.read_sql_query("SELECT * FROM insumos", conn)
        return df
    except sqlite3.OperationalError as e:
        print(f"Erro operacional: {e}")

# Função para buscar insumos que vencerão antes de uma data estipulada
def buscar_insumos_vencimento(data_limite):
    try:
        query = "SELECT * FROM insumos WHERE validade < ?"
        df = pd.read_sql_query(query, conn, params=(data_limite,))
        return df
    except sqlite3.OperationalError as e:
        print(f"Erro operacional: {e}")

# Widgets para adicionar insumo
input_nome = pn.widgets.TextInput(name='Nome do insumo')
input_quantidade = pn.widgets.IntInput(name='Quantidade')
input_secretario_id = pn.widgets.IntInput(name='ID do secretário')
botao_adicionar = pn.widgets.Button(name='Adicionar Insumo')

# Widgets para entrada de insumo
input_id_entrada = pn.widgets.IntInput(name='ID do insumo')
input_quantidade_entrada = pn.widgets.IntInput(name='Quantidade')
botao_entrada = pn.widgets.Button(name='Entrada de Insumo')

# Widgets para saída de insumo
input_id_saida = pn.widgets.IntInput(name='ID do insumo')
input_quantidade_saida = pn.widgets.IntInput(name='Quantidade')
botao_saida = pn.widgets.Button(name='Saída de Insumo')

# Widget para entrada da data limite
input_data_limite = pn.widgets.DatePicker(name='Data Limite')

# Botão para atualizar tabela
botao_atualizar = pn.widgets.Button(name='Atualizar Tabela')

# Botão para buscar insumos
botao_buscar = pn.widgets.Button(name='Buscar Insumos')

# Área para exibir os resultados
output_area = pn.pane.DataFrame()

# Funções para os botões
def on_adicionar_clicked(event):
    adicionar_insumo(input_nome.value, input_quantidade.value, input_secretario_id.value)
    print("Insumo adicionado com sucesso!")

def on_entrada_clicked(event):
    entrada_insumo(input_id_entrada.value, input_quantidade_entrada.value)
    print("Entrada de insumo registrada com sucesso!")

def on_saida_clicked(event):
    saida_insumo(input_id_saida.value, input_quantidade_saida.value)
    print("Saída de insumo registrada com sucesso!")

def on_atualizar_clicked(event):
    df = atualizar_tabela()
    output_area.object = df

def on_buscar_clicked(event):
    data_limite = input_data_limite.value.strftime('%Y-%m-%d')
    df = buscar_insumos_vencimento(data_limite)
    output_area.object = df

# Associar funções aos botões
botao_adicionar.on_click(on_adicionar_clicked)
botao_entrada.on_click(on_entrada_clicked)
botao_saida.on_click(on_saida_clicked)
botao_atualizar.on_click(on_atualizar_clicked)
botao_buscar.on_click(on_buscar_clicked)

# Layout do painel
painel = pn.Column(
    pn.Row(input_nome, input_quantidade, input_secretario_id, botao_adicionar),
    pn.Row(input_id_entrada, input_quantidade_entrada, botao_entrada),
    pn.Row(input_id_saida, input_quantidade_saida, botao_saida),
    pn.Row(input_data_limite, botao_buscar),
    botao_atualizar,
    output_area
)

# Servir o painel
painel.servable()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Column
    [0] Row
        [0] TextInput(name='Nome do insumo')
        [1] IntInput(name='Quantidade')
        [2] IntInput(name='ID do secretário')
        [3] Button(name='Adicionar Insumo')
    [1] Row
        [0] IntInput(name='ID do insumo')
        [1] IntInput(name='Quantidade')
        [2] Button(name='Entrada de Insumo')
    [2] Row
        [0] IntInput(name='ID do insumo')
        [1] IntInput(name='Quantidade')
        [2] Button(name='Saída de Insumo')
    [3] Row
        [0] DatePicker(name='Data Limite')
        [1] Button(name='Buscar Insumos')
    [4] Button(name='Atualizar Tabela')
    [5] DataFrame(None)

Support for third party widgets will remain active for the duration of the session. To disable support: